In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import re

# --- 1. データベースの準備 ---
db_name = 'google_repository.db'
path = '' 

try:
    conn = sqlite3.connect(path + db_name)
    cur = conn.cursor()
    
    # ※テーブルを一旦リセットしたい場合はコメントアウトを外してください
    # cur.execute("DROP TABLE IF EXISTS repository")

    sql = '''
    CREATE TABLE IF NOT EXISTS repository (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT, 
        main_language TEXT, 
        stars INTEGER
    );
    '''
    cur.execute(sql)
    conn.commit()
    
except sqlite3.Error as e:
    print('DB初期化エラー:', e)
    exit()

# --- 2. スクレイピングと保存 (全件取得モード) ---
print("Googleの全リポジトリのスクレイピングを開始します...")
print("※件数が多いため時間がかかります。中断する場合は Ctrl+C を押してください。")

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}

page = 1
total_count = 0

while True:
    url = f"https://github.com/orgs/google/repositories?page={page}"
    print(f'--- {page}ページ目 取得中... (現在: {total_count}件) ---')
    
    try:
        res = requests.get(url, headers=headers)
        
        if res.status_code != 200:
            print(f"アクセス失敗: {res.status_code}")
            break
            
        soup = BeautifulSoup(res.text, "html.parser")
        
        # リポジトリのリスト要素を取得
        repos = soup.find_all('li', class_=re.compile("Box-row|ListItem-module__listItem"))

        # 終了条件: リポジトリが見つからない場合
        if not repos:
            print("これ以上リポジトリが見つかりません。スクレイピングを終了します。")
            break

        for repo in repos:
            try:
                # --- 名前 ---
                name_tag = repo.find('a', itemprop="name codeRepository")
                if not name_tag:
                    name_tag = repo.find('a', class_=re.compile("Title-module__anchor"))
                name = name_tag.text.strip() if name_tag else "Unknown"

                # --- 言語 ---
                lang_tag = repo.find('span', itemprop="programmingLanguage")
                if not lang_tag:
                    lang_tag = repo.find(class_=re.compile("ReposListItem-module__Text|LanguageComponent"))
                language = lang_tag.text.strip() if lang_tag else "None"

                # --- スター数 ---
                stars_count = 0
                star_tag = repo.select_one('a[href$="/stargazers"]')
                if star_tag:
                    stars_text = star_tag.get_text().strip()
                    stars_text = re.sub(r'\s+', '', stars_text)
                    try:
                        if 'k' in stars_text:
                            stars_count = int(float(stars_text.replace('k', '')) * 1000)
                        else:
                            stars_count = int(stars_text.replace(',', ''))
                    except ValueError:
                        stars_count = 0

                # --- DB保存 ---
                cur.execute('INSERT INTO repository (name, main_language, stars) VALUES (?, ?, ?)', 
                            (name, language, stars_count))
                
                total_count += 1

            except Exception as e:
                print(f"解析エラー(個別): {e}")

            # 待機 (要件)
            time.sleep(1)
            
        conn.commit()
        page += 1

    except Exception as e:
        print(f'ページ処理エラー: {e}')
        break

# --- 3. 全データの表示 (ご要望箇所) ---
print("\n" + "="*30)
print("【全件表示】DBに保存されたすべてのデータを表示します")
print("="*30)

try:
    # 制限(LIMIT)なしで全件取得
    cur.execute('SELECT * FROM repository')
    rows = cur.fetchall()

    if not rows:
        print("データがありません。")
    else:
        # 見やすく整形して出力
        print(f"{'ID':<6} | {'Name':<40} | {'Language':<15} | {'Stars'}")
        print("-" * 80)
        
        for row in rows:
            r_id = row[0]
            # 名前が長すぎる場合に備えて調整（表示用）
            r_name = (row[1][:37] + '..') if len(row[1]) > 37 else row[1]
            r_lang = (row[2][:12] + '..') if len(row[2]) > 12 else row[2]
            r_stars = row[3]
            
            print(f"{r_id:<6} | {r_name:<40} | {r_lang:<15} | {r_stars}")
            
    print("-" * 80)
    print(f"合計表示件数: {len(rows)} 件")

except sqlite3.Error as e:
    print("SELECT実行エラー:", e)

# --- 終了処理 ---
conn.close()
print('\nすべての処理が完了しました。')

Googleの全リポジトリのスクレイピングを開始します...
※件数が多いため時間がかかります。中断する場合は Ctrl+C を押してください。
--- 1ページ目 取得中... (現在: 0件) ---
--- 2ページ目 取得中... (現在: 30件) ---
--- 3ページ目 取得中... (現在: 60件) ---
--- 4ページ目 取得中... (現在: 90件) ---
--- 5ページ目 取得中... (現在: 120件) ---
--- 6ページ目 取得中... (現在: 150件) ---
--- 7ページ目 取得中... (現在: 180件) ---
--- 8ページ目 取得中... (現在: 210件) ---
--- 9ページ目 取得中... (現在: 240件) ---
--- 10ページ目 取得中... (現在: 270件) ---
--- 11ページ目 取得中... (現在: 300件) ---
--- 12ページ目 取得中... (現在: 330件) ---
--- 13ページ目 取得中... (現在: 360件) ---
--- 14ページ目 取得中... (現在: 390件) ---
--- 15ページ目 取得中... (現在: 420件) ---
--- 16ページ目 取得中... (現在: 450件) ---
--- 17ページ目 取得中... (現在: 480件) ---
--- 18ページ目 取得中... (現在: 510件) ---
--- 19ページ目 取得中... (現在: 540件) ---
--- 20ページ目 取得中... (現在: 570件) ---
--- 21ページ目 取得中... (現在: 600件) ---
--- 22ページ目 取得中... (現在: 630件) ---
--- 23ページ目 取得中... (現在: 660件) ---
--- 24ページ目 取得中... (現在: 690件) ---
--- 25ページ目 取得中... (現在: 720件) ---
--- 26ページ目 取得中... (現在: 750件) ---
--- 27ページ目 取得中... (現在: 780件) ---
--- 28ページ目 取得中... (現在: 810件) ---


KeyboardInterrupt: 